In [4]:
import os
import zipfile


zip_path = "python-3.13-docs-text.zip"
extract_dir = "python_docs"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ ZIP extracted to:", extract_dir)


✅ ZIP extracted to: python_docs


In [5]:
from langchain.docstore.document import Document

documents = []

for root, _, files in os.walk("python_docs"):
    for file in files:
        if file.endswith(".txt"):
            with open(os.path.join(root, file), "r", encoding="utf-8") as f:
                text = f.read()
                documents.append(Document(page_content=text, metadata={"source": file}))

print(f"📄 Loaded {len(documents)} text documents.")


📄 Loaded 507 text documents.


In [6]:
#split documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

print(f"✂️ Total text chunks: {len(split_docs)}")


✂️ Total text chunks: 15919


In [7]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from getpass import getpass
import os

# Ask for API key safely
os.environ["OPENAI_API_KEY"] = getpass("🔐 Enter your OpenAI API key: ")

# Generate embeddings
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


🔐 Enter your OpenAI API key:  ········


C:\Users\trahu\AppData\Local\Temp\ipykernel_14220\3704368007.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


C:\Users\trahu\AppData\Local\Temp\ipykernel_14220\2928076918.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [10]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True,
    output_key="answer"  # ✅ tell it to store only the answer
)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    output_key="answer"  # ✅ same fix here
)


C:\Users\trahu\AppData\Local\Temp\ipykernel_14220\682399099.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
print("🤖 Ask me anything about Python docs! Type 'exit' to quit.\n")

while True:
    query = input("🧑 You: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("👋 Goodbye!")
        break

    result = conversation_chain({"question": query})

    print("\n🤖 Answer:\n" + result["answer"])

    print("\n📚 Sources:")
    for doc in result["source_documents"]:
        print(" -", doc.metadata["source"])
    
    print("\n---")


🤖 Ask me anything about Python docs! Type 'exit' to quit.



🧑 You:  how can I print matrix in python? 



🤖 Answer:
To print a matrix in Python, you can use a loop to iterate over each row and then print each element in that row. Here's an example using a 2D list as a matrix:

```python
# Define a 2D list as a matrix
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Print the matrix
for row in matrix:
    print(row)
```

When you run this code, it will print each row of the matrix on a new line. You can adjust the formatting as needed based on your requirements.

📚 Sources:
 - programming.txt
 - windows.txt
 - 3.0.txt
 - 2.0.txt

---


🧑 You:  can you solve a leetcode using python?



🤖 Answer:
I don't know.

📚 Sources:
 - programming.txt
 - programming.txt
 - introduction.txt
 - ios.txt

---


...


🤖 Answer:
Sure, here is a simple example of an if-else statement in Python:

```python
x = 10

if x > 5:
    print("x is greater than 5")
else:
    print("x is less than or equal to 5")
```

In this example, if the value of `x` is greater than 5, it will print "x is greater than 5"; otherwise, it will print "x is less than or equal to 5".

📚 Sources:
 - controlflow.txt
 - programming.txt
 - 3.10.txt
 - errors.txt

---


🧑 You:  do it for java



🤖 Answer:
I don't know.

📚 Sources:
 - controlflow.txt
 - 3.10.txt
 - programming.txt
 - compound_stmts.txt

---


🧑 You:  quit


👋 Goodbye!
